# Guidelines for Prompting
This is developed to serve as a cheatsheet consisting of best practices of Prompt Engineering.

Content from [**ChatGPT Prompt Engineering for Developers**](https://learn.deeplearning.ai/courses/chatgpt-prompt-eng/lesson/1/introduction) short course in Deeplearning.ai platform has been used for creating this cheatsheet.

We can save our API Key in "api_key.txt" file within Setup Files. Here we have used Together AI API Key as it has free credits for initial use. For playing around, this is sufficient. Refer -> https://api.together.xyz/

In [36]:
#To save Together API Key as an environment variable
import os

file_path = 'Setup Files/api_key.txt'

with open(file_path, 'r') as file:
    contents = file.read()

os.environ["TOGETHER_API_KEY"] = contents

### Helper function

Here we are using `meta-llama/Llama-Vision-Free` model.

For implementing using OpenAI, you can refer [chat completions endpoint](https://platform.openai.com/docs/guides/chat). 

In [121]:
from together import Together

model = "meta-llama/Llama-Vision-Free"
client = Together(api_key = os.getenv("TOGETHER_API_KEY"))

def get_completion(prompt, temperature=0, model=model):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output; 0 deterministic; 1 creative
    )
    return response.choices[0].message.content


## L1: Prompting Principles
Basically there are two prompting principles.

- **Principle 1: Write clear and specific instructions**
- **Principle 2: Give the model time to “think”**

They need to be followed in order to write effective prompts for large language models(LLMs).

### Principle 1: Write clear and specific instructions

#### Tactic 1: Use delimiters to clearly indicate distinct parts of the input
- Delimiters can be anything like: ```, """, < >, `<tag> </tag>`, `:`

In [38]:
text = f"""
You should express what you want a model to do by \ 
providing instructions that are as clear and \ 
specific as you can possibly make them. \ 
This will guide the model towards the desired output, \ 
and reduce the chances of receiving irrelevant \ 
or incorrect responses. Don't confuse writing a \ 
clear prompt with writing a short prompt. \ 
In many cases, longer prompts provide more clarity \ 
and context for the model, which can lead to \ 
more detailed and relevant outputs.
"""
prompt = f"""
Summarize the text delimited by triple backticks \ 
into a single sentence.
```{text}```
"""
response = get_completion(prompt)
print(response)

To get accurate and relevant responses from a model, provide clear and specific instructions in a prompt that is as detailed as possible.


#### Tactic 2: Ask for a structured output
- JSON, HTML

In [39]:
prompt = f"""
Generate a list of three made-up book titles along \ 
with their authors and genres. 
Provide them in JSON format with the following keys: 
book_id, title, author, genre.
"""
response = get_completion(prompt)
print(response)

Here are three made-up book titles in JSON format:

```
[
  {
    "book_id": "BK001",
    "title": "The Starlight Serenaders",
    "author": "Ava Winters",
    "genre": "Fantasy"
  },
  {
    "book_id": "BK002",
    "title": "The Memory Weaver",
    "author": "Ethan Blackwood",
    "genre": "Science Fiction"
  },
  {
    "book_id": "BK003",
    "title": "The Whispering Walls of Willowdale",
    "author": "Lila Rose",
    "genre": "Mystery"
  }
]
```


#### Tactic 3: Ask the model to check whether conditions are satisfied

In [40]:
text_1 = f"""
Making a cup of tea is easy! First, you need to get some \ 
water boiling. While that's happening, \ 
grab a cup and put a tea bag in it. Once the water is \ 
hot enough, just pour it over the tea bag. \ 
Let it sit for a bit so the tea can steep. After a \ 
few minutes, take out the tea bag. If you \ 
like, you can add some sugar or milk to taste. \ 
And that's it! You've got yourself a delicious \ 
cup of tea to enjoy.
"""
prompt = f"""
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, \ 
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \ 
then simply write \"No steps provided.\"

\"\"\"{text_1}\"\"\"
"""
response = get_completion(prompt)
print("Completion for Text 1:")
print(response)

Completion for Text 1:
Step 1 - Get some water boiling.
Step 2 - Grab a cup and put a tea bag in it.
Step 3 - Pour the hot water over the tea bag.
Step 4 - Let the tea steep for a few minutes.
Step 5 - Take out the tea bag.
Step 6 - Add sugar or milk to taste (optional).
Step 7 - Enjoy your delicious cup of tea.


In [41]:
text_2 = f"""
The sun is shining brightly today, and the birds are \
singing. It's a beautiful day to go for a \ 
walk in the park. The flowers are blooming, and the \ 
trees are swaying gently in the breeze. People \ 
are out and about, enjoying the lovely weather. \ 
Some are having picnics, while others are playing \ 
games or simply relaxing on the grass. It's a \ 
perfect day to spend time outdoors and appreciate the \ 
beauty of nature.
"""
prompt = f"""
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, \ 
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \ 
then simply write \"No steps provided.\"

\"\"\"{text_2}\"\"\"
"""
response = get_completion(prompt)
print("Completion for Text 2:")
print(response)

Completion for Text 2:
No steps provided.


#### Tactic 4: "Few-shot" prompting

- Providing an example of response before posting the query. This will help in getting a structured response in the format in which we require.

In [42]:
prompt = f"""
Your task is to answer in a consistent style.

<child>: Teach me about patience.

<grandparent>: The river that carves the deepest \ 
valley flows from a modest spring; the \ 
grandest symphony originates from a single note; \ 
the most intricate tapestry begins with a solitary thread.

<child>: Teach me about resilience.
"""
response = get_completion(prompt)
print(response)

<grandparent>: The lotus flower blooms in the muddiest of waters, yet its petals remain unsoiled. The tree that bends in the storm is not broken, but strengthened. The river that flows around the boulder finds its way to the sea.


### Principle 2: Give the model time to “think” 

#### Tactic 1: Specify the steps required to complete a task

In [43]:
text = f"""
In a charming village, siblings Jack and Jill set out on \ 
a quest to fetch water from a hilltop \ 
well. As they climbed, singing joyfully, misfortune \ 
struck—Jack tripped on a stone and tumbled \ 
down the hill, with Jill following suit. \ 
Though slightly battered, the pair returned home to \ 
comforting embraces. Despite the mishap, \ 
their adventurous spirits remained undimmed, and they \ 
continued exploring with delight.
"""
# example 1
prompt_1 = f"""
Perform the following actions: 
1 - Summarize the following text delimited by triple \
backticks with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the following \
keys: french_summary, num_names.

Separate your answers with line breaks.

Text:
```{text}```
"""
response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)

Completion for prompt 1:
Here is a 1 sentence summary of the text:

Jack and Jill, two siblings, embark on a quest to fetch water from a hilltop well, but their adventure is temporarily derailed by a mishap.

La traduction de la résumé en français est :
Jack et Jill, deux frères et sœurs, partent en quête d'eau d'un puits situé au sommet d'une colline, mais leur aventure est temporairement déroutée par un accident.

Les noms mentionnés dans la résumé français sont :
- Jack
- Jill

Voici un objet JSON contenant les clés "french_summary" et "num_names" :
```
{
  "french_summary": "Jack et Jill, deux frères et sœurs, partent en quête d'eau d'un puits situé au sommet d'une colline, mais leur aventure est temporairement déroutée par un accident.",
  "num_names": 2
}
```


#### Ask for output in a specified format

In [44]:
prompt_2 = f"""
Your task is to perform the following actions: 
1 - Summarize the following text delimited by 
  <> with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the 
  following keys: french_summary, num_names.

Use the following format:
Text: <text to summarize>
Summary: <summary>
Translation: <summary translation>
Names: <list of names in summary>
Output JSON: <json with summary and num_names>

Text: <{text}>
"""
response = get_completion(prompt_2)
print("\nCompletion for prompt 2:")
print(response)


Completion for prompt 2:
Text: <In a charming village, siblings Jack and Jill set out on \ a quest to fetch water from a hilltop \ well. As they climbed, singing joyfully, misfortune \ struck—Jack tripped on a stone and tumbled \ down the hill, with Jill following suit. \ Though slightly battered, the pair returned home to \ comforting embraces. Despite the mishap, \ their adventurous spirits remained undimmed, and they \ continued exploring with delight.>

Summary: The siblings Jack and Jill embarked on a quest to fetch water from a hilltop well, but their adventure was marked by a mishap.

Translation: Les frères et sœurs Jack et Jill ont entrepris une quête pour aller chercher de l'eau d'un puits situé en haut d'une colline, mais leur aventure a été marquée par un accident.

Names: Jack, Jill

Output JSON: 
{
  "french_summary": "Les frères et sœurs Jack et Jill ont entrepris une quête pour aller chercher de l'eau d'un puits situé en haut d'une colline, mais leur aventure a été mar

#### Tactic 2: Instruct the model to work out its own solution before rushing to a conclusion

In [45]:
prompt = f"""
Determine if the student's solution is correct or not.

Question:
I'm building a solar power installation and I need \
 help working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \ 
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations 
as a function of the number of square feet.

Student's Solution:
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
"""
response = get_completion(prompt)
print(response)

The student's solution is mostly correct, but there is a small mistake.

The student correctly identified the costs:

1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 10x (not 100x)

The total cost should be the sum of these costs:

Total cost = 100x + 250x + 100,000 + 10x
= 360x + 100,000

So, the correct total cost as a function of the number of square feet is 360x + 100,000.


- The student's solution is actually not correct. The models might miss them out if it rushes to generate a response.
- We can fix this by instructing the model to work out its own solution first.

In [46]:
prompt = f"""
Your task is to determine if the student's solution \
is correct or not.
To solve the problem do the following:
- First, work out your own solution to the problem including the final total. 
- Then compare your solution to the student's solution \ 
and evaluate if the student's solution is correct or not. 
Don't decide if the student's solution is correct until 
you have done the problem yourself.

Use the following format:
Question:
```
question here
```
Student's solution:
```
student's solution here
```
Actual solution:
```
steps to work out the solution and your solution here
```
Is the student's solution the same as actual solution \
just calculated:
```
yes or no
```
Student grade:
```
correct or incorrect
```

Question:
```
I'm building a solar power installation and I need help \
working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations \
as a function of the number of square feet.
``` 
Student's solution:
```
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
```
Actual solution:
"""
response = get_completion(prompt)
print(response)

Question:
```
I'm building a solar power installation and I need help working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost me a flat $100k per year, and an additional $10 / square foot
What is the total cost for the first year of operations as a function of the number of square feet.
```
Student's solution:
```
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
```
Actual solution:
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 10x (not 100x, as the problem states an additional $10/square foot)
Total cost: 100x + 250x + 100,000 + 10x = 360x + 100,000

Is the student's solution the same as actual solution just calculated:
no

Student grade:


## Model Limitations: Hallucinations
- Boie is a real company, the product name is not real.
- But the model generates a response as if the product exists. This is called Hallucination and is a known issue in LLM models which could be overcome to an extent by fine tuning.

In [47]:
prompt = f"""
Tell me about AeroGlide UltraSlim Smart Toothbrush by Boie
"""
response = get_completion(prompt)
print(response)

The AeroGlide UltraSlim Smart Toothbrush by Boie is a high-tech electric toothbrush designed for a superior oral care experience. Here are some of its key features:

**Design and Ergonomics**

The AeroGlide UltraSlim has a sleek and slim design, making it easy to maneuver in tight spaces and comfortable to hold. Its ergonomic shape fits perfectly in the hand, allowing for precise control and reduced fatigue during brushing.

**Smart Technology**

This toothbrush features advanced smart technology, including:

1. **Bluetooth Connectivity**: Connects to the Boie app (available for iOS and Android) to track brushing habits, receive personalized feedback, and monitor progress.
2. **Real-Time Feedback**: Provides instant feedback on brushing technique, ensuring a thorough and effective clean.
3. **Customizable Brushing Plans**: Create and schedule personalized brushing plans based on your oral health needs and goals.

**Oral Care Features**

The AeroGlide UltraSlim offers:

1. **5 Modes**: 

## L2: Iterative Prompt Development
In this lesson, we'll iteratively analyze and refine your prompts to generate marketing copy from a product fact sheet.

### Generate a marketing product description from a product fact sheet

In [48]:
fact_sheet_chair = """
OVERVIEW
- Part of a beautiful family of mid-century inspired office furniture, 
including filing cabinets, desks, bookcases, meeting tables, and more.
- Several options of shell color and base finishes.
- Available with plastic back and front upholstery (SWC-100) 
or full upholstery (SWC-110) in 10 fabric and 6 leather options.
- Base finish options are: stainless steel, matte black, 
gloss white, or chrome.
- Chair is available with or without armrests.
- Suitable for home or business settings.
- Qualified for contract use.

CONSTRUCTION
- 5-wheel plastic coated aluminum base.
- Pneumatic chair adjust for easy raise/lower action.

DIMENSIONS
- WIDTH 53 CM | 20.87”
- DEPTH 51 CM | 20.08”
- HEIGHT 80 CM | 31.50”
- SEAT HEIGHT 44 CM | 17.32”
- SEAT DEPTH 41 CM | 16.14”

OPTIONS
- Soft or hard-floor caster options.
- Two choices of seat foam densities: 
 medium (1.8 lb/ft3) or high (2.8 lb/ft3)
- Armless or 8 position PU armrests 

MATERIALS
SHELL BASE GLIDER
- Cast Aluminum with modified nylon PA6/PA66 coating.
- Shell thickness: 10 mm.
SEAT
- HD36 foam

COUNTRY OF ORIGIN
- Italy
"""

In [49]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)


Here's a product description based on the technical specifications:

**Elevate Your Workspace with the Mid-Century Inspired Office Chair**

Add a touch of sophistication to your home or business setting with our stylish office chair, part of a beautiful family of mid-century inspired office furniture. Available in a range of shell colors and base finishes, including stainless steel, matte black, gloss white, and chrome, this chair is designed to complement any décor.

Choose from two upholstery options: plastic back and front upholstery (SWC-100) or full upholstery (SWC-110) in 10 fabric and 6 leather options. The chair's 5-wheel plastic coated aluminum base provides stability and durability, while the pneumatic chair adjust allows for easy height adjustment.

With a range of options to suit your needs, this chair is perfect for both home and business settings. Choose from soft or hard-floor caster options, two seat foam densities (medium or high), and armless or 8-position PU armrests

### Issue 1: The text is too long 
- Limit the number of words/sentences/characters.

In [50]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)


Here is a 50-word product description based on the technical specifications:

"Elevate your workspace with our stylish, mid-century inspired office chair. Available in various shell colors and base finishes, with options for plastic or full upholstery, armrests, and seat foam density. Suitable for home or business, this chair is built to last with a durable cast aluminum base and HD36 foam seat."


In [52]:
#Length might vary in LLMs from what is specified as generation is dynamic. 
# But it will be approximately closer to the specified length.
len(response.split())

62

### Issue 2. Text focuses on the wrong details
- Ask it to focus on the aspects that are relevant to the intended audience.

In [54]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

At the end of the description, include every 7-character 
Product ID in the technical specification.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

Here is a 50-word product description for a furniture retailer:

"Introducing our mid-century inspired office chair, crafted from durable cast aluminum with a modified nylon PA6/PA66 coating and 10mm shell thickness. Upholstered in HD36 foam, this chair features a 5-wheel plastic coated aluminum base and pneumatic adjust for easy use. Available in various finishes and upholstery options."

Product IDs: SWC-100, SWC-110


### Issue 3. Description needs a table of dimensions
- Ask it to extract information and organize it in a table.

In [55]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

At the end of the description, include every 7-character 
Product ID in the technical specification.

After the description, include a table that gives the 
product's dimensions. The table should have two columns.
In the first column include the name of the dimension. 
In the second column include the measurements in inches only.

Give the table the title 'Product Dimensions'.

Format everything as HTML that can be used in a website. 
Place the description in a <div> element.

Technical specifications: ```{fact_sheet_chair}```
"""

response = get_completion(prompt)
print(response)

Here is the HTML code for the product description and dimensions table:

```html
<div>
  Our stylish and durable office chair is part of a beautiful family of mid-century inspired office furniture. Constructed from high-quality materials, this chair features a 5-wheel plastic coated aluminum base and a pneumatic chair adjust for easy raise/lower action. The shell is made from cast aluminum with a modified nylon PA6/PA66 coating, providing a durable and long-lasting finish. The seat is made from HD36 foam, ensuring optimal comfort and support. Available with or without armrests, this chair is suitable for both home and business settings and is qualified for contract use.

  The following Product IDs are available for this product: SWC-100, SWC-110, SWC-100, SWC-110, SWC-100, SWC-110, SWC-100.
</div>

<table>
  <tr>
    <th>Product Dimensions</th>
  </tr>
  <tr>
    <td>WIDTH</td>
    <td>20.87"</td>
  </tr>
  <tr>
    <td>DEPTH</td>
    <td>20.08"</td>
  </tr>
  <tr>
    <td>HEIGHT</td>

### Load Python libraries to view HTML

In [56]:
from IPython.display import display, HTML

In [57]:
display(HTML(response))

## L3: Summarizing
In this lesson, we will summarize text with a focus on specific topics.

### Text to summarize

In [58]:
prod_review = """
Got this panda plush toy for my daughter's birthday, \
who loves it and takes it everywhere. It's soft and \ 
super cute, and its face has a friendly look. It's \ 
a bit small for what I paid though. I think there \ 
might be other options that are bigger for the \ 
same price. It arrived a day earlier than expected, \ 
so I got to play with it myself before I gave it \ 
to her.
"""

### Summarize with a word/sentence/character limit

In [59]:
prompt = f"""
Your task is to generate a short summary of a product \
review from an ecommerce site. 

Summarize the review below, delimited by triple 
backticks, in at most 30 words. 

Review: ```{prod_review}```
"""

response = get_completion(prompt)
print(response)


```
The reviewer's daughter loves the soft and cute panda plush toy, but finds it small for the price. Arrived a day early.
```


### Summarize with a focus on price and value

In [61]:
prompt = f"""
Your task is to generate a short summary of a product \
review from an ecommerce site to give feedback to the \
pricing deparmtment, responsible for determining the \
price of the product.  

Summarize the review below, delimited by triple 
backticks, in at most 30 words, and focusing on any aspects \
that are relevant to the price and perceived value. 

Review: ```{prod_review}```
"""

response = get_completion(prompt)
print(response)


```
The reviewer found the plush toy soft and cute, but felt it was too small for the price. Perceived value could be improved with larger options at the same price.
```


### Try "extract" instead of "summarize"
- Summaries include topics that are not related to the topic of focus.

In [62]:
prompt = f"""
Your task is to extract relevant information from \ 
a product review from an ecommerce site to give \
feedback to the Shipping department. 

From the review below, delimited by triple quotes \
extract the information relevant to shipping and \ 
delivery. Limit to 30 words. 

Review: ```{prod_review}```
"""

response = get_completion(prompt)
print(response)

Here is the relevant information for the Shipping department:

```
It arrived a day earlier than expected.
```
This indicates that the shipping was faster than expected, which is a positive experience for the customer.


### Summarize multiple product reviews

In [63]:

review_1 = prod_review 

# review for a standing lamp
review_2 = """
Needed a nice lamp for my bedroom, and this one \
had additional storage and not too high of a price \
point. Got it fast - arrived in 2 days. The string \
to the lamp broke during the transit and the company \
happily sent over a new one. Came within a few days \
as well. It was easy to put together. Then I had a \
missing part, so I contacted their support and they \
very quickly got me the missing piece! Seems to me \
to be a great company that cares about their customers \
and products. 
"""

# review for an electric toothbrush
review_3 = """
My dental hygienist recommended an electric toothbrush, \
which is why I got this. The battery life seems to be \
pretty impressive so far. After initial charging and \
leaving the charger plugged in for the first week to \
condition the battery, I've unplugged the charger and \
been using it for twice daily brushing for the last \
3 weeks all on the same charge. But the toothbrush head \
is too small. I’ve seen baby toothbrushes bigger than \
this one. I wish the head was bigger with different \
length bristles to get between teeth better because \
this one doesn’t.  Overall if you can get this one \
around the $50 mark, it's a good deal. The manufactuer's \
replacements heads are pretty expensive, but you can \
get generic ones that're more reasonably priced. This \
toothbrush makes me feel like I've been to the dentist \
every day. My teeth feel sparkly clean! 
"""

# review for a blender
review_4 = """
So, they still had the 17 piece system on seasonal \
sale for around $49 in the month of November, about \
half off, but for some reason (call it price gouging) \
around the second week of December the prices all went \
up to about anywhere from between $70-$89 for the same \
system. And the 11 piece system went up around $10 or \
so in price also from the earlier sale price of $29. \
So it looks okay, but if you look at the base, the part \
where the blade locks into place doesn’t look as good \
as in previous editions from a few years ago, but I \
plan to be very gentle with it (example, I crush \
very hard items like beans, ice, rice, etc. in the \ 
blender first then pulverize them in the serving size \
I want in the blender then switch to the whipping \
blade for a finer flour, and use the cross cutting blade \
first when making smoothies, then use the flat blade \
if I need them finer/less pulpy). Special tip when making \
smoothies, finely cut and freeze the fruits and \
vegetables (if using spinach-lightly stew soften the \ 
spinach then freeze until ready for use-and if making \
sorbet, use a small to medium sized food processor) \ 
that you plan to use that way you can avoid adding so \
much ice if at all-when making your smoothie. \
After about a year, the motor was making a funny noise. \
I called customer service but the warranty expired \
already, so I had to buy another one. FYI: The overall \
quality has gone done in these types of products, so \
they are kind of counting on brand recognition and \
consumer loyalty to maintain sales. Got it in about \
two days.
"""

reviews = [review_1, review_2, review_3, review_4]



In [64]:
for i in range(len(reviews)):
    prompt = f"""
    Your task is to generate a short summary of a product \ 
    review from an ecommerce site. 

    Summarize the review below, delimited by triple \
    backticks in at most 20 words. 

    Review: ```{reviews[i]}```
    """

    response = get_completion(prompt)
    print(i, response, "\n")


0 ```
Soft, cute panda plush toy, but small for price; arrived early, daughter loves it.
``` 

1 ```
```
Lamp arrived quickly, with minor issues resolved promptly by company, demonstrating good customer service and product quality.
```
```
 

2 ```
Impressive battery life, but small toothbrush head is a drawback; good deal around $50, with affordable generic replacement heads.
``` 

3 ```
Disappointed with price hike and quality decline, but still functional blender with some tips for optimal use.
``` 



## L4: Inferring
In this lesson, we will infer sentiment and topics from product reviews and news articles.

### Product review text

In [65]:
lamp_review = """
Needed a nice lamp for my bedroom, and this one had \
additional storage and not too high of a price point. \
Got it fast.  The string to our lamp broke during the \
transit and the company happily sent over a new one. \
Came within a few days as well. It was easy to put \
together.  I had a missing part, so I contacted their \
support and they very quickly got me the missing piece! \
Lumina seems to me to be a great company that cares \
about their customers and products!!
"""

### Sentiment (positive/negative)

In [66]:
prompt = f"""
What is the sentiment of the following product review, 
which is delimited with triple backticks?

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

The sentiment of this product review is overwhelmingly **POSITIVE**. The reviewer is extremely satisfied with their purchase, citing several reasons:

* The lamp met their needs and was reasonably priced.
* The company provided excellent customer service, replacing a broken part and sending a new lamp quickly.
* The support team was responsive and helpful in resolving an issue with a missing part.
* The reviewer feels that the company, Lumina, genuinely cares about its customers and products.

The tone is enthusiastic and appreciative, with the reviewer using phrases like "happily sent over a new one" and "very quickly got me the missing piece!" to emphasize their positive experience.


In [67]:
prompt = f"""
What is the sentiment of the following product review, 
which is delimited with triple backticks?

Give your answer as a single word, either "positive" \
or "negative".

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

Positive


### Identify types of emotions

In [68]:
prompt = f"""
Identify a list of emotions that the writer of the \
following review is expressing. Include no more than \
five items in the list. Format your answer as a list of \
lower-case words separated by commas.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

gratitude, relief, satisfaction, trust, appreciation


### Identify anger

In [69]:
prompt = f"""
Is the writer of the following review expressing anger?\
The review is delimited with triple backticks. \
Give your answer as either yes or no.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

No


### Extract product and company name from customer reviews

In [70]:
prompt = f"""
Identify the following items from the review text: 
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Item" and "Brand" as the keys. 
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.
  
Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

```json
{
  "Item": "lamp",
  "Brand": "Lumina"
}
```


### Doing multiple tasks at once

In [71]:
prompt = f"""
Identify the following items from the review text: 
- Sentiment (positive or negative)
- Is the reviewer expressing anger? (true or false)
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Sentiment", "Anger", "Item" and "Brand" as the keys.
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.
Format the Anger value as a boolean.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

```
{
  "Sentiment": "positive",
  "Anger": false,
  "Item": "lamp",
  "Brand": "Lumina"
}
```


### Inferring topics

In [72]:
story = """
In a recent survey conducted by the government, 
public sector employees were asked to rate their level 
of satisfaction with the department they work at. 
The results revealed that NASA was the most popular 
department with a satisfaction rating of 95%.

One NASA employee, John Smith, commented on the findings, 
stating, "I'm not surprised that NASA came out on top. 
It's a great place to work with amazing people and 
incredible opportunities. I'm proud to be a part of 
such an innovative organization."

The results were also welcomed by NASA's management team, 
with Director Tom Johnson stating, "We are thrilled to 
hear that our employees are satisfied with their work at NASA. 
We have a talented and dedicated team who work tirelessly 
to achieve our goals, and it's fantastic to see that their 
hard work is paying off."

The survey also revealed that the 
Social Security Administration had the lowest satisfaction 
rating, with only 45% of employees indicating they were 
satisfied with their job. The government has pledged to 
address the concerns raised by employees in the survey and 
work towards improving job satisfaction across all departments.
"""

### Infer 5 topics

In [80]:
prompt = f"""
Determine five topics that are being discussed in the \
following text, which is delimited by triple backticks.

Make each item one or two words long. 

Format your response as a list of items separated by commas.\
Just include only the topics in the response.

Text sample: '''{story}'''
"""
response = get_completion(prompt)
print(response)

NASA satisfaction, Government survey, Job satisfaction, Employee morale, Department rankings


In [81]:
response.split(sep=',')

['NASA satisfaction',
 ' Government survey',
 ' Job satisfaction',
 ' Employee morale',
 ' Department rankings']

### Make a news alert for certain topics

In [83]:
topic_list = [
    "nasa", "local government", "engineering", 
    "employee satisfaction", "federal government"
]

In [96]:
prompt = f"""
Determine whether each item in the following list of \
topics is a topic in the text below, which
is delimited with triple backticks.

Give your answer as follows:
item from the list: 0 or 1

Return the result in dictionary datatype. 
Only have the answer, don't add extra words.
List of topics: {", ".join(topic_list)}

Text sample: '''{story}'''
"""
response = get_completion(prompt)
print(response)

{
    "nasa": 1,
    "local government": 0,
    "engineering": 0,
    "employee satisfaction": 1,
    "federal government": 1
}


## L5: Transforming

In this lesson, we will explore how to use Large Language Models for text transformation tasks such as language translation, spelling and grammar checking, tone adjustment, and format conversion.

### Translation

In [98]:
prompt = f"""
Translate the following English text to Spanish: \ 
```Hi, I would like to order a blender```
"""
response = get_completion(prompt)
print(response)

Here is the translation:

"Hola, me gustaría pedir un licuadora"


In [100]:
prompt = f"""
Tell me in English which language this is: 
```Combien coûte le lampadaire?```
"""
response = get_completion(prompt)
print(response)

This is French.


In [103]:
prompt = f"""
Translate the following  text to French and Spanish
and English (like a pirate slang): \
```I want to order a basketball```
"""
response = get_completion(prompt)
print(response)

Here are the translations:

**French:** Je veux commander un panier de basket.

**Spanish:** Quiero pedir un balón de baloncesto.

**Pirate slang (English):** Arrr, I be wantin' to charter a hoop ball, matey!


In [104]:
prompt = f"""
Translate the following text to Spanish in both the \
formal and informal forms: 
'Would you like to order a pillow?'
"""
response = get_completion(prompt)
print(response)

Here are the translations:

**Formal:**
¿Le gustaría pedir un cojín?

**Informal:**
¿Quieres pedir un cojín?

Note: In Spanish, the formal and informal forms of "you" (usted and tú) are used to address the person, but the verb conjugation remains the same. The difference lies in the pronoun used to address the person.


### Universal Translator
Imagine you are in charge of IT at a large multinational e-commerce company. Users are messaging you with IT issues in all their native languages. Your staff is from all over the world and speaks only their native languages. You need a universal translator!

In [105]:
user_messages = [
  "La performance du système est plus lente que d'habitude.",  # System performance is slower than normal         
  "Mi monitor tiene píxeles que no se iluminan.",              # My monitor has pixels that are not lighting
  "Il mio mouse non funziona",                                 # My mouse is not working
  "Mój klawisz Ctrl jest zepsuty",                             # My keyboard has a broken control key
  "我的屏幕在闪烁"                                               # My screen is flashing
] 

In [107]:
for issue in user_messages:
    prompt = f"Tell me what language this is in English: ```{issue}```"
    lang = get_completion(prompt)
    print(f"Original message ({lang}): {issue}")

    prompt = f"""
    Translate the following  text to English : ```{issue}```
    """
    response = get_completion(prompt)
    print(response, "\n")

Original message (This text is in French.): La performance du système est plus lente que d'habitude.
La traduction du texte en anglais est :

"The system's performance is slower than usual." 

Original message (This sentence is in Spanish.): Mi monitor tiene píxeles que no se iluminan.
The translation of the text is:

"My monitor has pixels that don't light up." 

Original message (The language is Italian.): Il mio mouse non funziona
The translation of the text is:

"My mouse doesn't work." 

Original message (The language is Polish.): Mój klawisz Ctrl jest zepsuty
The translation of the text is:

"My Ctrl key is broken" 

Original message (This is in Chinese.): 我的屏幕在闪烁
"My screen is flickering." 



### Tone Transformation
Writing can vary based on the intended audience.


In [108]:
prompt = f"""
Translate the following from slang to a business letter: 
'Dude, This is Joe, check out this spec on this standing lamp.'
"""
response = get_completion(prompt)
print(response)

Here is a translation of the slang into a formal business letter:

"Dear [Recipient],

I am writing to bring to your attention a specification for a standing lamp that I would like to share with you.

Sincerely,
Joe"

Or, if you want to make it a bit more polished:

"Dear [Recipient],

I am pleased to share with you a detailed specification for a standing lamp that I believe would be of interest to you.

Please find the specification attached.

Best regards,
Joe"

Note that I've removed the informal greeting ("Dude") and the casual tone, and replaced it with a more formal greeting and closing. I've also removed the phrase "check out this spec" and replaced it with a more formal phrase, such as "I would like to share with you" or "I am pleased to share with you".


### Format Conversion
ChatGPT can translate between formats. The prompt should describe the input and output formats.

In [111]:
data_json = { "resturant employees" :[ 
    {"name":"Shyam", "email":"shyamjaiswal@gmail.com"},
    {"name":"Bob", "email":"bob32@gmail.com"},
    {"name":"Jai", "email":"jai87@gmail.com"}
]}

prompt = f"""
Translate the following dictionary from JSON to an HTML \
table with column headers and title. I want the response in HTML format: {data_json}
"""
response = get_completion(prompt)
print(response)

Here is the HTML table representation of the given JSON data:

```html
<!DOCTYPE html>
<html>
<head>
    <title>Restaurant Employees</title>
    <style>
        table {
            border-collapse: collapse;
        }
        th, td {
            border: 1px solid black;
            padding: 5px;
        }
    </style>
</head>
<body>
    <h1>Restaurant Employees</h1>
    <table>
        <tr>
            <th>Name</th>
            <th>Email</th>
        </tr>
        <tr>
            <td>Shyam</td>
            <td>shyamjaiswal@gmail.com</td>
        </tr>
        <tr>
            <td>Bob</td>
            <td>bob32@gmail.com</td>
        </tr>
        <tr>
            <td>Jai</td>
            <td>jai87@gmail.com</td>
        </tr>
    </table>
</body>
</html>
```

However, if you want to generate this HTML dynamically from a JSON string, you can use JavaScript. Here is an example:

```javascript
const jsonData = {
    'resturant employees': [
        {'name': 'Shyam', 'email': 'shyamjaisw

In [112]:
from IPython.display import display, Markdown, Latex, HTML, JSON
display(HTML(response))

### Spellcheck/Grammar check.

Here are some examples of common grammar and spelling problems and the LLM's response. 

To signal to the LLM that you want it to proofread your text, you instruct the model to 'proofread' or 'proofread and correct'.

In [113]:
text = [ 
  "The girl with the black and white puppies have a ball.",  # The girl has a ball.
  "Yolanda has her notebook.", # ok
  "Its going to be a long day. Does the car need it’s oil changed?",  # Homonyms
  "Their goes my freedom. There going to bring they’re suitcases.",  # Homonyms
  "Your going to need you’re notebook.",  # Homonyms
  "That medicine effects my ability to sleep. Have you heard of the butterfly affect?", # Homonyms
  "This phrase is to cherck chatGPT for speling abilitty"  # spelling
]
for t in text:
    prompt = f"""Proofread and correct the following text
    and rewrite the corrected version. If you don't find
    and errors, just say "No errors found". Don't use 
    any punctuation around the text:
    ```{t}```"""
    response = get_completion(prompt)
    print(response)

The corrected version is:
The girl with the black and white puppies has a ball.

I corrected "have" to "has" because "has" is the correct verb to use when referring to a singular subject (the girl) that possesses something (the ball).
No errors found
Here is the corrected version:

Its going to be a long day. Does the car need its oil changed?

I corrected the following errors:

* "it's" is a contraction of "it is" or "it has", whereas "its" is a possessive pronoun. In this sentence, "its" is the correct choice because it shows that the car owns or possesses the oil.
Here is the corrected version:

Their goes my freedom. They're going to bring their suitcases.

I corrected the following errors:

* "There" should be "They're" (contraction of "they are")
* "they're" should be "their" (possessive pronoun)
* "suitcases" was correct, but I added an apostrophe to make it clear that it's a plural noun.
Your going to need your notebook.

(I corrected "you're" to "your", which is the correct us

In [114]:
text = f"""
Got this for my daughter for her birthday cuz she keeps taking \
mine from my room.  Yes, adults also like pandas too.  She takes \
it everywhere with her, and it's super soft and cute.  One of the \
ears is a bit lower than the other, and I don't think that was \
designed to be asymmetrical. It's a bit small for what I paid for it \
though. I think there might be other options that are bigger for \
the same price.  It arrived a day earlier than expected, so I got \
to play with it myself before I gave it to my daughter.
"""
prompt = f"proofread and correct this review: ```{text}```"
response = get_completion(prompt)
print(response)

Here is the proofread and corrected review:

"I got this for my daughter's birthday because she keeps taking mine from my room. Yes, adults also like pandas too! She takes it everywhere with her, and it's super soft and adorable. One of the ears is a bit lower than the other, and I don't think that was designed to be asymmetrical. It's a bit smaller than I expected for the price, and I think there might be other options that are bigger for the same price. I was pleasantly surprised when it arrived a day earlier than expected, so I got to play with it myself before giving it to my daughter."

I made the following corrections:

* Added a period after "room" to complete the sentence
* Changed "cuz" to "because" to make the language more formal and correct
* Changed "super soft and cute" to "super soft and adorable" to use a more descriptive and appealing adjective
* Changed "I don't think that was designed to be asymmetrical" to a more concise and clear sentence
* Changed "It's a bit smal

In [116]:
from redlines import Redlines

diff = Redlines(text,response)
display(Markdown(diff.output_markdown))

<span style='color:red;font-weight:700;text-decoration:line-through;'>Got </span><span style='color:green;font-weight:700;'>Here is the proofread and corrected review: </span>

<span style='color:green;font-weight:700;'>"I got </span>this for my <span style='color:red;font-weight:700;text-decoration:line-through;'>daughter for her </span><span style='color:green;font-weight:700;'>daughter's </span>birthday <span style='color:red;font-weight:700;text-decoration:line-through;'>cuz </span><span style='color:green;font-weight:700;'>because </span>she keeps taking mine from my <span style='color:red;font-weight:700;text-decoration:line-through;'>room.  </span><span style='color:green;font-weight:700;'>room. </span>Yes, adults also like pandas <span style='color:red;font-weight:700;text-decoration:line-through;'>too.  </span><span style='color:green;font-weight:700;'>too! </span>She takes it everywhere with her, and it's super soft and <span style='color:red;font-weight:700;text-decoration:line-through;'>cute.  </span><span style='color:green;font-weight:700;'>adorable. </span>One of the ears is a bit lower than the other, and I don't think that was designed to be asymmetrical. It's a bit <span style='color:red;font-weight:700;text-decoration:line-through;'>small for what I paid for it though. </span><span style='color:green;font-weight:700;'>smaller than I expected for the price, and </span>I think there might be other options that are bigger for the same <span style='color:red;font-weight:700;text-decoration:line-through;'>price.  It </span><span style='color:green;font-weight:700;'>price. I was pleasantly surprised when it </span>arrived a day earlier than expected, so I got to play with it myself before <span style='color:green;font-weight:700;'>giving it to my daughter." </span>

<span style='color:green;font-weight:700;'>I made the following corrections: </span>

<span style='color:green;font-weight:700;'>* Added a period after "room" to complete the sentence </span>

<span style='color:green;font-weight:700;'>* Changed "cuz" to "because" to make the language more formal and correct </span>

<span style='color:green;font-weight:700;'>* Changed "super soft and cute" to "super soft and adorable" to use a more descriptive and appealing adjective </span>

<span style='color:green;font-weight:700;'>* Changed "I don't think that was designed to be asymmetrical" to a more concise and clear sentence </span>

<span style='color:green;font-weight:700;'>* Changed "It's a bit small for what I paid for it though" to "It's a bit smaller than I expected for the price" to make the language more concise and clear </span>

<span style='color:green;font-weight:700;'>* Changed "I think there might be other options that are bigger for the same price" to a more concise and clear sentence </span>

<span style='color:green;font-weight:700;'>* Changed "I got to play with it myself before </span>I gave it to my <span style='color:red;font-weight:700;text-decoration:line-through;'>daughter.</span><span style='color:green;font-weight:700;'>daughter" to "before giving it to my daughter" to make the language more concise and clear.</span>

In [117]:
prompt = f"""
proofread and correct this review. Make it more compelling. 
Ensure it follows APA style guide and targets an advanced reader. 
Output in markdown format.
Text: ```{text}```
"""
response = get_completion(prompt)
display(Markdown(response))

**Corrected and Enhanced Review**
=====================================

**Title:** A Cuddly Companion for the Young and Young-at-Heart: A Review of the Panda Plush Toy

**Rating:** 3.5/5 stars

As a parent, I was thrilled to gift my daughter a panda plush toy for her birthday, as she had been eyeing mine for quite some time. What I didn't expect was that I, too, would fall in love with this adorable creature. The soft and cuddly texture of the panda is undeniable, and its cute design makes it a perfect companion for both children and adults alike.

**Design and Quality**
--------------------

While the panda's overall design is charming, I did notice that one of its ears is slightly lower than the other. Upon closer inspection, I suspect that this may not be an intentional design choice, but rather a minor manufacturing defect. Nevertheless, this imperfection does not detract from the overall appeal of the toy.

**Size and Value**
-----------------

One area where the panda plush toy falls short is its size. For the price I paid, I had expected a larger toy, but unfortunately, this one is on the smaller side. Upon further research, I discovered that there may be other options available in the market that offer a similar product at a comparable price point, albeit with a larger size.

**Shipping and Delivery**
-------------------------

I was pleasantly surprised to receive the panda plush toy a day earlier than expected, which gave me the opportunity to play with it myself before gifting it to my daughter. The shipping and delivery process was smooth and efficient, and I appreciate the attention to detail in getting the product to me ahead of schedule.

**Conclusion**
----------

In conclusion, while the panda plush toy may have its minor flaws, its charm and appeal make it a delightful companion for both children and adults. If you're in the market for a soft and cuddly toy, I would recommend considering this option, but be aware of the potential size limitations.

## L6: Expanding
In this lesson, we will generate customer service emails that are tailored to each customer's review.

### Customize the automated reply to a customer email

In [118]:
# given the sentiment from the lesson on "inferring",
# and the original customer message, customize the email
sentiment = "negative"

# review for a blender
review = f"""
So, they still had the 17 piece system on seasonal \
sale for around $49 in the month of November, about \
half off, but for some reason (call it price gouging) \
around the second week of December the prices all went \
up to about anywhere from between $70-$89 for the same \
system. And the 11 piece system went up around $10 or \
so in price also from the earlier sale price of $29. \
So it looks okay, but if you look at the base, the part \
where the blade locks into place doesn’t look as good \
as in previous editions from a few years ago, but I \
plan to be very gentle with it (example, I crush \
very hard items like beans, ice, rice, etc. in the \ 
blender first then pulverize them in the serving size \
I want in the blender then switch to the whipping \
blade for a finer flour, and use the cross cutting blade \
first when making smoothies, then use the flat blade \
if I need them finer/less pulpy). Special tip when making \
smoothies, finely cut and freeze the fruits and \
vegetables (if using spinach-lightly stew soften the \ 
spinach then freeze until ready for use-and if making \
sorbet, use a small to medium sized food processor) \ 
that you plan to use that way you can avoid adding so \
much ice if at all-when making your smoothie. \
After about a year, the motor was making a funny noise. \
I called customer service but the warranty expired \
already, so I had to buy another one. FYI: The overall \
quality has gone done in these types of products, so \
they are kind of counting on brand recognition and \
consumer loyalty to maintain sales. Got it in about \
two days.
"""

In [119]:
prompt = f"""
You are a customer service AI assistant.
Your task is to send an email reply to a valued customer.
Given the customer email delimited by ```, \
Generate a reply to thank the customer for their review.
If the sentiment is positive or neutral, thank them for \
their review.
If the sentiment is negative, apologize and suggest that \
they can reach out to customer service. 
Make sure to use specific details from the review.
Write in a concise and professional tone.
Sign the email as `AI customer agent`.
Customer review: ```{review}```
Review sentiment: {sentiment}
"""
response = get_completion(prompt)
print(response)

Subject: Re: Feedback on Recent Purchase

Dear valued customer,

Thank you for taking the time to share your thoughts on your recent purchase. I apologize for the disappointing experience you had with the blender's price increase and the perceived decrease in overall quality. I understand that the price hike from $49 to $70-$89 was unexpected, and I apologize for any frustration this may have caused.

I also apologize for the issue with the motor making a funny noise after a year of use. Although the warranty had expired, I appreciate your feedback on the quality of the product. I will pass on your comments to our product development team to help improve future models.

If you have any further concerns or would like to discuss this matter further, please don't hesitate to reach out to our customer service team directly. They will be happy to assist you.

Thank you again for your feedback. We value your loyalty and appreciate your business.

Best regards,
AI customer agent


### Remind the model to use details from the customer's email

In [122]:
prompt = f"""
You are a customer service AI assistant.
Your task is to send an email reply to a valued customer.
Given the customer email delimited by ```, \
Generate a reply to thank the customer for their review.
If the sentiment is positive or neutral, thank them for \
their review.
If the sentiment is negative, apologize and suggest that \
they can reach out to customer service. 
Make sure to use specific details from the review.
Write in a concise and professional tone.
Sign the email as `AI customer agent`.
Customer review: ```{review}```
Review sentiment: {sentiment}
"""
response = get_completion(prompt, temperature=0.7)
print(response)

Subject: Re: Feedback on Recent Purchase

Dear valued customer,

Thank you for taking the time to share your feedback on your recent purchase. I regret to hear that our product did not meet your expectations in terms of quality and price. I apologize for the price increase you experienced in December, which, as you mentioned, seemed excessive. I also understand your concerns about the design of the blender's base, which may not be as robust as previous models.

I'm sorry to hear that the motor made a funny noise after a year, and I apologize for the inconvenience this caused. Although the warranty had expired, I appreciate your feedback on the overall quality of our product, and I will pass it on to our quality control team.

As a valued customer, I want to ensure that you have an exceptional experience with our brand. If you have any further concerns or issues, please don't hesitate to reach out to our customer service team directly. We're here to help and will do our best to resolve 

## Exercise: Creating a Custom Chatbot

### The Chat Format

In this exercise, we will explore how we can utilize the chat format to have extended conversations with chatbots personalized or specialized for specific tasks or behaviors.

In [132]:
def get_completion_from_messages(messages, temperature=0, model=model):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output; 0 deterministic; 1 creative
    )
    return response.choices[0].message.content

In [133]:
messages =  [  
{'role':'system', 'content':'You are an assistant that speaks like Shakespeare. Be precise'},    
{'role':'user', 'content':'tell me a joke'},   
{'role':'assistant', 'content':'Why did the chicken cross the road'},   
{'role':'user', 'content':'I don\'t know'}  ]

In [134]:
response = get_completion_from_messages(messages, temperature=0)
print(response)

Good sir, 'tis a jest most wondrous and fine,
To cross the road, the chicken didst design,
To get to the other side, where eggs didst shine,
And thus, 'twas said, "To get away from the farmer's whine!"


In [135]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Hi, my name is Isa'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Hi Isa! Nice to meet you! Is there anything I can help you with or would you like to chat about something in particular?


In [136]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Yes,  can you remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

I'm happy to help! However, I don't actually know your name since we just started chatting. I'm a friendly chatbot, but I don't have the ability to retain information about individual users across conversations. Would you like to tell me your name?


In [137]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Hi, my name is Isa'},
{'role':'assistant', 'content': "Hi Isa! It's nice to meet you. \
Is there anything I can help you with today?"},
{'role':'user', 'content':'Yes, you can remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

I remember! Your name is Isa! How's your day going so far?


### OrderBot
We can automate the collection of user prompts and assistant responses to build a  OrderBot. The OrderBot will take orders at a pizza restaurant. 

The below code might give errors. Refer [here](https://s172-30-107-92p8888.lab-aws-production.deeplearning.ai/notebooks/l8_chatbot/l8-chatbot.ipynb) for a similar functional chatbot.

In [138]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context) 
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))
 
    return pn.Column(*panels)


In [ ]:
import panel as pn  # GUI
pn.extension()

panels = [] # collect display 

context = [ {'role':'system', 'content':"""
You are OrderBot, an automated service to collect orders for a pizza restaurant. \
You first greet the customer, then collects the order, \
and then asks if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very conversational friendly style. \
The menu includes \
pepperoni pizza  12.95, 10.00, 7.00 \
cheese pizza   10.95, 9.25, 6.50 \
eggplant pizza   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
greek salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
sausage 3.00 \
canadian bacon 3.50 \
AI sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
bottled water 5.00 \
"""} ]  # accumulate messages


inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

In [ ]:
messages =  context.copy()
messages.append(
{'role':'system', 'content':'create a json summary of the previous food order. Itemize the price for each item\
 The fields should be 1) pizza, include size 2) list of toppings 3) list of drinks, include size   4) list of sides include size  5)total price '},    
)
 #The fields should be 1) pizza, price 2) list of toppings 3) list of drinks, include size include price  4) list of sides include size include price, 5)total price '},    

response = get_completion_from_messages(messages, temperature=0)
print(response)

#### These are the generic guidelines to be followed for Effective Prompt Generation
#### This is important to get required responses from the LLMs.

## Reference:

https://writingprompts.com/bad-grammar-examples/
